In [1]:
import crm.ads_database as ads_database

# ads_database.set(user_id="test", content="helloworld")

In [2]:
ads_database.push(user_id="test", content="helloworld")

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000203'), 'opTime': {'ts': Timestamp(1726566446, 6), 't': 515}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1726566446, 7), 'signature': {'hash': b'\xa0\xd2\xaa\xd0\xf9_\xb2\xa0`A\x8a\xba}WP\xf027\xf1\x92', 'keyId': 7366617972116291585}}, 'operationTime': Timestamp(1726566446, 6), 'updatedExisting': True}, acknowledged=True)

In [12]:
from datetime import datetime

type(datetime.now())

datetime.datetime